In [ ]:
!rm *.png

In [ ]:
!rm -rf FashionDataset

In [ ]:
!git clone https://github.com/szx159753/FashionDataset.git

Cloning into 'FashionDataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [ ]:
!mv FashionDataset/focal_loss.py /content/

In [ ]:
!tar xvf FashionDataset/data.tar

In [ ]:
from __future__ import print_function, division
import os

from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from collections import defaultdict
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import transforms
import subprocess
from focal_loss import FocalLoss

In [ ]:
data_dir='FashionDataset'
classes=[7,3,3,4,6,3]
# files={}
# labels={}
# for dir in ['train','val']:
#     files[dir]=open(os.path.join(data_dir,'split/'+dir+'.txt')).read().split('\n')
#     labels[dir]=open(os.path.join(data_dir,'split/'+dir+'_attr.txt')).read().split('\n')

In [ ]:
class MyDataset(Dataset):
    def __init__(self,dir,y_label=False):
        self.transform = transforms.Compose([
            transforms.Resize([224,224]),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.files=open(os.path.join(data_dir,'split/'+dir+'.txt')).read().split('\n')[0:-1]
        self.len=len(self.files)
        self.y_label=y_label

        bbox=open(os.path.join(data_dir,'split/'+dir+'_bbox.txt')).read().split('\n')[0:-1]
        bbox=[bb.split(' ') for bb in bbox]
        self.bbox=bbox
        if self.y_label is True:
            labels=open(os.path.join(data_dir,'split/'+dir+'_attr.txt')).read().split('\n')[0:-1]
            labels=[li.split(' ') for li in labels]
            self.labels=labels
    def __getitem__(self, idx):
        img_obj = Image.open(os.path.join(data_dir,self.files[idx]))
        bbox = np.array([int(l) for l in self.bbox[idx]],dtype=np.int)
        img_obj = img_obj.crop([bbox[0],bbox[1],bbox[2],bbox[3]])
        img_obj=self.transform(img_obj)
        if self.y_label is True:
            labels = np.array([int(l) for l in self.labels[idx]],dtype=np.float32)
            labels = torch.from_numpy(labels)
            return img_obj,labels
        else:
            return img_obj,self.files[idx]
    def __len__(self):
        return len(self.files)

In [ ]:
image_datasets={x:MyDataset(x,y)
          for x,y in [['train',True],['val',True],['test',False]]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=y, num_workers=2)
              for x,y in [['train',True],['val',True],['test',False]]}

#device=torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, label_num,state,num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        phases=['train', 'val']
        if state=='iter':
          phases=['train']
        elif state=='final':
          phases=['train', 'val']

        
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels=labels[:,label_num]
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                #torch.save(model.state_dict(), "state_for_class_" + str(label_num) + ".pth")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def test_model(model):
    since = time.time()
    model.eval()  # Set model to evaluate mode
    predictions=torch.tensor([]).to(device)
        # Iterate over data.
    for inputs,file in dataloaders['test']:
        inputs = inputs.to(device)
        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions=torch.cat([predictions,preds],0)
    return predictions

In [ ]:
def train(model_conv,class_num,state,epoch,lr):

  num_ftrs = model_conv.fc.in_features
  model_conv.fc = nn.Linear(num_ftrs, classes[class_num])
  model_conv = model_conv.to(device)

  criterion = FocalLoss()

  # Observe that only parameters of final layer are being optimized as
  # opposed to before.
  #optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
  optimizer_conv = optim.SGD(model_conv.parameters(), lr=lr, momentum=0.7)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
  model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler,class_num,state, num_epochs=epoch)
  torch.save(model_conv.state_dict(),state+"_state_for_class_"+str(class_num)+".pth")
  return model_conv

In [ ]:
def test(model,class_num):
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, classes[class_num])
  model.load_state_dict(torch.load("final_state_for_class_"+str(class_num)+".pth"))
  model = model.to(device)

  # Observe that only parameters of final layer are being optimized as
  # opposed to before.
  # optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
  # optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

  # Decay LR by a factor of 0.1 every 7 epochs
  # exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
  labels = test_model(model)
  return labels

In [ ]:
iter=5
iter_epoch=1
model = torchvision.models.resnet50(pretrained=True)
for i in range(iter):
  for class_num in range(len(classes)):
    model = train(model,class_num,'iter',iter_epoch,lr=1)

torch.save(model.state_dict(), "state_after_iter" + ".pth")

In [ ]:
epoch=30
model = torchvision.models.resnet50(pretrained=True)
for class_num in range(len(classes)):
  # num_ftrs = model.fc.in_features
  # model.fc = nn.Linear(num_ftrs, classes[-1])
  # model.load_state_dict(torch.load("state_after_iter" + ".pth"))
  
  model = train(model,class_num,'final',epoch,lr=0.0001)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Epoch 0/29
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 1.3519 Acc: 0.5318
val Loss: 1.0955 Acc: 0.6290

Epoch 1/29
----------
train Loss: 1.1172 Acc: 0.6228
val Loss: 0.9475 Acc: 0.6990

Epoch 2/29
----------
train Loss: 1.0160 Acc: 0.6670
val Loss: 0.8721 Acc: 0.7230

Epoch 3/29
----------
train Loss: 0.9456 Acc: 0.6848
val Loss: 0.8437 Acc: 0.7130

Epoch 4/29
----------


In [ ]:
model = torchvision.models.resnet50(pretrained=True)
result=[]
for class_num in range(len(classes)):
  predictions=test(model,class_num)
  result.append(predictions.cpu().numpy())
result=np.matrix(result)
result=result.T
np.savetxt('prediction.txt', np.c_[result],
 fmt='%d',delimiter='\t')


In [ ]:
!tar -cvf state.tar *.pth

final_state_for_class_0.pth
final_state_for_class_1.pth
final_state_for_class_2.pth
final_state_for_class_3.pth
final_state_for_class_4.pth
final_state_for_class_5.pth
